In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

sentence = "Repeat is the best medicine for memory".split()

vocab = list(set(sentence))
print(vocab)

['memory', 'for', 'the', 'Repeat', 'is', 'best', 'medicine']


In [7]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0
print(word2index)

{'memory': 1, 'for': 2, 'the': 3, 'Repeat': 4, 'is': 5, 'best': 6, 'medicine': 7, '<unk>': 0}


In [8]:
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'memory', 2: 'for', 3: 'the', 4: 'Repeat', 5: 'is', 6: 'best', 7: 'medicine', 0: '<unk>'}


In [9]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

X, Y = build_data(sentence, word2index)
print(np.shape(X),X)
print(np.shape(Y),Y)


torch.Size([1, 6]) tensor([[4, 5, 3, 6, 7, 2]])
torch.Size([1, 6]) tensor([[5, 3, 6, 7, 2, 1]])


In [10]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [11]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [12]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [13]:
output = model(X)
print(output)

tensor([[ 0.0391,  0.0269, -0.1966, -0.2213,  0.0553,  0.0201,  0.0203, -0.0671],
        [ 0.1800,  0.2479,  0.0547,  0.5066,  0.0090,  0.1353, -0.2105, -0.1792],
        [ 0.3324, -0.0466, -0.3398, -0.2580,  0.0098, -0.1296, -0.1854, -0.1141],
        [ 0.4105, -0.0122,  0.0660, -0.0608, -0.1456, -0.0724,  0.2037, -0.2946],
        [ 0.0479,  0.0138, -0.2413,  0.0986, -0.0626,  0.2433, -0.1859, -0.1129],
        [ 0.3295, -0.0074, -0.1281,  0.1938, -0.0932,  0.2086, -0.1492, -0.3010]],
       grad_fn=<ViewBackward>)


In [14]:
print(output.shape)

torch.Size([6, 8])


In [15]:
decode = lambda y: [index2word.get(x) for x in y]

In [16]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.1225 
Repeat Repeat the <unk> <unk> is <unk>

[41/201] 1.4613 
Repeat is the best medicine for the

[81/201] 0.8339 
Repeat is the best medicine for memory

[121/201] 0.4278 
Repeat is the best medicine for memory

[161/201] 0.2250 
Repeat is the best medicine for memory

[201/201] 0.1285 
Repeat is the best medicine for memory

